In [1]:
%pip install mediapipe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Import the required libraries

In [2]:
import mediapipe as mp

In [3]:
import cv2
import numpy as np
from collections import deque

Different arrays to handle colour points

In [4]:
bluept = [deque(maxlen = 1024)]
greenpt = [deque(maxlen = 1024)]
redpt = [deque(maxlen = 1024)]
yellowpt = [deque(maxlen = 1024)]

In [5]:
blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

Kernel : a purpose of dilatation

In [6]:
kernel = np.ones((5,5) , np.uint8)

In [7]:
colors = [(255, 0 , 0) , (0, 255, 0) , (0 , 0, 255) , (0, 255, 255)]
color_index = 0

For Canvas window

In [12]:
whiteboard = np.zeros((471, 636, 3)) + 255
whiteboard = cv2.rectangle(whiteboard, (40, 1), (140, 65), (0,0,0), 2)
whiteboard = cv2.rectangle(whiteboard, (160, 1), (255, 65), colors[0], -1)
whiteboard = cv2.rectangle(whiteboard, (275, 1), (370, 65), colors[1], -1)
whiteboard = cv2.rectangle(whiteboard, (390, 1), (485, 65), colors[2], -1)
whiteboard = cv2.rectangle(whiteboard, (505, 1), (600, 65), colors[3], -1)

In [13]:
cv2.putText(whiteboard, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(whiteboard, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(whiteboard, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(whiteboard, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(whiteboard, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.namedWindow('Whiteboard', cv2.WINDOW_AUTOSIZE)

For initialising mediapipe 

In [14]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands = 1, min_detection_confidence = 0.7)
mp_draw = mp.solutions.drawing_utils

For operating webcam

In [15]:
cap = cv2.VideoCapture(0)
ret = True
while ret:
    ret, frame = cap.read()

    x, y, c = frame.shape

    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.rectangle(frame, (40,1), (140,65), (0,0,0), 2)
    frame = cv2.rectangle(frame, (160,1), (255,65), colors[0], -1)
    frame = cv2.rectangle(frame, (275,1), (370,65), colors[1], -1)
    frame = cv2.rectangle(frame, (390,1), (485,65), colors[2], -1)
    frame = cv2.rectangle(frame, (505,1), (600,65), colors[3], -1)
    
    cv2.putText(frame, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

    result = hands.process(framergb)

    if result.multi_hand_landmarks:
        landmarks = []

        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                lm_x = int(lm.x * 640)
                lm_y = int(lm.y * 480)

                landmarks.append([lm_x , lm_y])

            mp_draw.draw_landmarks(frame, handslms, mp_hands.HAND_CONNECTIONS)

        fore_finger = (landmarks[8][0],landmarks[8][1])
        center = fore_finger
        thumb = (landmarks[4][0],landmarks[4][1])
        cv2.circle(frame, center, 3, (0,255,0),-1)
        print(center[1]-thumb[1])   

        if (thumb[1]-center[1]<30):
            bluept.append(deque(maxlen=512))
            blue_index += 1
            greenpt.append(deque(maxlen=512))
            green_index += 1
            redpt.append(deque(maxlen=512))
            red_index += 1
            yellowpt.append(deque(maxlen=512))
            yellow_index += 1  

        elif center[1] <= 65:
            if 40 <= center[0] <= 140: # Clear Button
                bluept = [deque(maxlen=512)]
                greenpt = [deque(maxlen=512)]
                redpt = [deque(maxlen=512)]
                yellowpt = [deque(maxlen=512)]

                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0

                whiteboard[67:,:,:] = 255
            elif 160 <= center[0] <= 255:
                    color_index = 0   #for Blue colour
            elif 275 <= center[0] <= 370:
                    color_index = 1   #for Green colour
            elif 390 <= center[0] <= 485:
                    color_index = 2   #for Red colour
            elif 505 <= center[0] <= 600:
                    color_index = 3   #for Yellow colour

        else :
            if color_index == 0:
                bluept[blue_index].appendleft(center)
            elif color_index == 1:
                greenpt[green_index].appendleft(center)
            elif color_index == 2:
                redpt[red_index].appendleft(center)
            elif color_index == 3:
                yellowpt[yellow_index].appendleft(center)
    else:
        bluept.append(deque(maxlen=512))
        blue_index += 1
        greenpt.append(deque(maxlen=512))
        green_index += 1
        redpt.append(deque(maxlen=512))
        red_index += 1
        yellowpt.append(deque(maxlen=512))
        yellow_index += 1  


# Draw lines of all the colors on the canvas and frame
    points = [bluept, greenpt, redpt, yellowpt]
    
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(whiteboard, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    cv2.imshow("Output", frame) 
    cv2.imshow("Paint", whiteboard)

    if cv2.waitKey(1) == ord('q'):
        break

# release the webcam and destroy all active windows
cap.release()
cv2.destroyAllWindows()


22
16
25
22
24
25
22
22
22
20
21
20
19
20
18
18
18
19
18
17
18
19
17
17
18
20
19
16
15
13
11
11
8
5
2
6
2
3
-2
-3
-8
-7
-5
-12
-12
-15
-15
-16
-11
-17
-14
-14
-12
-12
-16
-15
-13
-14
-12
-13
-17
-19
-22
-19
-17
-20
-19
-19
-20
-17
-21
-17
-20
-17
-19
-18
-17
-15
-14
-18
-16
-19
-17
-19
-17
-16
-15
-14
-12
-12
-13
-11
-13
-12
-14
-12
-12
-11
-14
-13
-13
-14
-14
-13
-15
-12
-13
-13
-14
-13
-14
-13
-14
-14
-14
-14
-14
-12
-14
-13
-15
-13
-15
-13
-13
-22
-48
-85
-95
-106
-116
-119
-122
-125
-126
-126
-127
-127
-128
-130
-129
-128
-129
-128
-129
-129
-128
-128
-127
-127
-127
-127
-126
-127
-126
-128
-126
-125
-122
-115
-101
-72
-36
-15
-13
-16
-15
-13
-15
-15
-15
-14
-15
-14
-15
-13
-14
-14
-15
-14
-15
-14
-14
-58
-25
-22
-15
-15
-14
-16
-15
-14
-32
-5
-22
-17
-25
-27
-29
-23
-20
-13
-16
-15
-15
-19
-57
-56
-88
-110
-116
-116
-118
-120
-121
-121
-124
-123
-124
-123
-125
-121
-123
-124
-124
-123
-125
-125
-125
-127
-127
-127
-127
-129
-127
-128
-128
-130
-132
-129
-115
-98
-73
-41
-15
-14
-9